In [1]:
# *basic
import os
import scripts.train_SpaEMo_utils
from scripts.train_SpaEMo_utils import * 
from importlib import reload
reload (scripts.train_SpaEMo_utils)

from pathlib import Path
import sys 
import torch.distributed
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs
from accelerate.utils import set_seed
from scripts.logger import setup_logger
from timm.optim import create_optimizer_v2
from timm.scheduler import  create_scheduler, create_scheduler_v2
import torch.multiprocessing as mp
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Test dataloader first

In [2]:
config = OmegaConf.load("configs/SpaEMo_P14_config.yaml")
output_dir = config.experiment.output_dir
accelerator = Accelerator(
        gradient_accumulation_steps=config.training.gradient_accumulation_steps,
        mixed_precision=config.training.mixed_precision,
        split_batches=False,

    )
logger = setup_logger(name=config.experiment.project, log_level="INFO",
        output_file=f"{output_dir}/log{accelerator.process_index}.txt")

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=False)
  warnings.warn(


In [3]:
print(config.model.tokenizer)
tokenizer = AutoTokenizer.from_pretrained("facebook/m2m100_1.2B")
tokenizer.padding_side= 'right'
train_dataloader, dev_dataloader, test_dataloader = create_dataloader(config, 
                                                                          logger, 
                                                                          accelerator,
                                                                            tokenizer, device="cpu")

text/m2m_1.2b
[11/30 00:03:49 SpaEMo_P14]: Creating Signloaders. Batch_size = 8


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


train dataloader done!
dev dataloader done!
train dataloader done!


In [4]:
# Get a sample from the train dataloader
sample_batch = next(iter(train_dataloader))
# Print the sample batch
print(sample_batch)


({'name_batch': ['16March_2011_Wednesday_tagesschau-2219', '25February_2011_Friday_tagesschau-566', '03August_2010_Tuesday_tagesschau-2779', '10November_2010_Wednesday_tagesschau-6800', '27February_2011_Sunday_tagesschau-6113', '03September_2010_Friday_tagesschau-8427', '28August_2009_Friday_tagesschau-5873', '18November_2011_Friday_tagesschau-2150'], 'emo_batch': {'pixel_values': tensor([[[[0.5922, 0.5922, 0.5922,  ..., 0.5922, 0.5922, 0.5922],
          [0.5922, 0.5922, 0.5922,  ..., 0.5922, 0.5922, 0.5922],
          [0.5922, 0.5922, 0.5922,  ..., 0.5922, 0.5922, 0.5922],
          ...,
          [0.2078, 0.2078, 0.2078,  ..., 0.2235, 0.2235, 0.2235],
          [0.2078, 0.2078, 0.2078,  ..., 0.2235, 0.2235, 0.2235],
          [0.2078, 0.2078, 0.2078,  ..., 0.2235, 0.2235, 0.2235]],

         [[0.6549, 0.6549, 0.6549,  ..., 0.6627, 0.6627, 0.6627],
          [0.6549, 0.6549, 0.6549,  ..., 0.6627, 0.6627, 0.6627],
          [0.6549, 0.6549, 0.6549,  ..., 0.6627, 0.6627, 0.6627],
     

In [5]:
sample_batch[0].keys()


dict_keys(['name_batch', 'emo_batch', 'image_batch', 'clip_batch', 'num_frames_batch', 'num_clips_batch', 'path_batch'])

In [6]:
name_batch = sample_batch[0]['name_batch']
print(name_batch)

['16March_2011_Wednesday_tagesschau-2219', '25February_2011_Friday_tagesschau-566', '03August_2010_Tuesday_tagesschau-2779', '10November_2010_Wednesday_tagesschau-6800', '27February_2011_Sunday_tagesschau-6113', '03September_2010_Friday_tagesschau-8427', '28August_2009_Friday_tagesschau-5873', '18November_2011_Friday_tagesschau-2150']


In [7]:
emo_batch = sample_batch[0]['emo_batch']
print(emo_batch.pixel_values.shape)

torch.Size([1016, 3, 224, 224])


In [8]:
image_batch = sample_batch[0]['image_batch']
print(image_batch.pixel_values.shape)

torch.Size([1016, 3, 224, 224])


In [9]:
clip_batch = sample_batch[0]['clip_batch']
print(clip_batch.pixel_values.shape)

torch.Size([119, 16, 3, 224, 224])


In [10]:
num_frames = sample_batch[0]['num_frames_batch']
print(num_frames)
print(sum(num_frames))

[224, 152, 112, 96, 104, 88, 144, 96]
1016


In [11]:
num_clips = sample_batch[0]['num_clips_batch']
print(sum(num_clips))
print(num_clips)

119
[27, 18, 13, 11, 12, 10, 17, 11]


In [12]:
path_batch  = sample_batch[0]['path_batch']
print(path_batch[1])

['images0001.png', 'images0002.png', 'images0003.png', 'images0004.png', 'images0005.png', 'images0006.png', 'images0007.png', 'images0008.png', 'images0009.png', 'images0010.png', 'images0011.png', 'images0012.png', 'images0013.png', 'images0014.png', 'images0015.png', 'images0016.png', 'images0017.png', 'images0018.png', 'images0019.png', 'images0020.png', 'images0021.png', 'images0022.png', 'images0023.png', 'images0024.png', 'images0025.png', 'images0026.png', 'images0027.png', 'images0028.png', 'images0029.png', 'images0030.png', 'images0031.png', 'images0032.png', 'images0033.png', 'images0034.png', 'images0035.png', 'images0036.png', 'images0037.png', 'images0038.png', 'images0039.png', 'images0040.png', 'images0041.png', 'images0042.png', 'images0043.png', 'images0044.png', 'images0045.png', 'images0046.png', 'images0047.png', 'images0048.png', 'images0049.png', 'images0050.png', 'images0051.png', 'images0052.png', 'images0053.png', 'images0054.png', 'images0055.png', 'images00

In [13]:
sample_batch[1].keys()

dict_keys(['input_ids', 'attention_mask'])

In [14]:
sample_batch[1]['input_ids']

tensor([[128022,   1807,    257,   4414,   9431,    152,  19959,  10945,    788,
            569,  15215,    223,   8156,    286,    127,   8156,  69726,    152,
          40762,   2583,  16137,    569,   3551,   1386,   3382,   4691,   2291,
           1672,    152, 111765,  26892,    569,     25,   3352,    286,  87059,
          21214,   3493,  46201, 111134,   1019,  62817,   1908,    286,   1279,
          87276,   1808,      2],
        [128022,    624,    208,  10365,    624,   1547,   1653,  45989,  91881,
          88392,  35568,     88,  34906,   2249,  92399,  37747,    788,     28,
             83,  47410,   6002,   4691,   2246,    421, 111564,  22021,  30412,
           3121,      2,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1],
        [128022,  25652,  94677,  88980,    485,   8857,   7061,    257,   4304,
           2535,   1115,   7061,    569, 

In [15]:
sample_batch[1]['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1

## Creating the model 

In [16]:
model = create_SpaEMo(config, logger)



[11/30 00:05:18 SpaEMo_P14]: Creating model and loss module.
MLP: self.in_features: 2560, hidden_features: 2560
trainable params: 4,718,592 || all params: 1,244,188,672 || trainable%: 0.3793


In [17]:
## note that current model is in adaptor type 1

src_input = sample_batch[0]
tgt_input = sample_batch[1]


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(device)

cuda


In [19]:
with torch.no_grad():
  output, temporal_features = model(src_input, tgt_input)


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\vit\modeling_vit.py:252: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(


In [41]:
#224, 152, 112, 96, 104, 88, 144, 96
print(temporal_features['visual_feat'][0].shape)
print(temporal_features['src_attn'][0])
print(sum(temporal_features['src_attn'][1]))

torch.Size([53, 2560])
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True])
tensor(35)


In [27]:
temporal_features['visual_feat'].shape

torch.Size([8, 53, 2560])

In [48]:
print(temporal_features['visual_feat'][1][52]) 
print(temporal_features['visual_feat'][1][51])
print(temporal_features['visual_feat'][1][50]) 


tensor([0.0000, 0.0000, 0.9718,  ..., 1.1052, 0.0000, 0.1996], device='cuda:0')
tensor([0.0000, 0.0000, 0.9718,  ..., 1.1052, 0.0000, 0.1996], device='cuda:0')
tensor([0.0000, 0.0000, 0.9718,  ..., 1.1052, 0.0000, 0.1996], device='cuda:0')
